In [ ]:
import pickle
import json
import os
import glob
import shutil
import time
import numpy as np
from pathlib import Path
from PIL import Image
from pprint import pprint
import torch

# 0. Naming convention

coco uses running number for the files, but we don't have this, so maybe like this:

| | origin dataset | subject | action | origin image | H | R | sub image | file extension |
| -- | -- | --- | --- | --- | --- | -- | -- | -- |
|Image:| 0dd | 0ss | 0aa | xxxxxx | hh | rr | i | .png |
|Anno: | 0dd | 0ss | 0aa | xxxxxx | hh | rr | i | .json |

**Example images:**

Human3.6M - S1 - Directions - frame 1055 - H1.2R1.0 - center: `001 001 001 001055 12 10 0.png`

Genebody - Ahha - NaN - frame 123 - H1.2R1.0 - east: `002 001 000 000123 12 10 1.png`

Genebody - Ahha - NaN - frame 85 - H1.8R1.0 - south: `002 001 000 000085 18 10 3.png`

**Annotations:**

For annotations the current convention is one .json file for each subject-action

Human3.6M - S1 - Directions: `001 001 001 xxxxxx 12 10 i.json`

Genebody - Ahha - NaN - all frames - H1.2R1.0 - all positions: `002 001 000 xxxxxx 12 10 i.json`

Genebody - Ahha - NaN - all frames - H1.0R0.5 - south: `002 001 000 xxxxxx 10 05 i.png`

# Y. Generate coco format pose annotations

In [ ]:
from gen_cocolike_anno import create_cocolike_pose_anno
import time

In [ ]:
genebody_subjects = ['ahha', 'alejandro', 'anastasia', 'aosilan', 'arslan',
                     'barlas', 'barry', 'camilo', 'dannier', 'gaoxing',
                     'huajiangtao5', 'joseph', 'kamal_ejaz', 'kemal',
                     'lihongyun', 'natacha', 'quyuanning', 'rabbi', 'rivera',
                     'songyujie', 'sunyuxing', 'wuwenyan', 'xujiarui',
                     'zhanghao', 'zhanghongwei', 'zhangziyu', 'zhuna2']
human36m_subjects = ['S1', 'S5', 'S6', 'S7', 'S8', 'S9', 'S11']
zjumocap_subjects = ['p313', 'p315', 'p377', 'p386', 'p387', 'p390', 'p392', 'p393', 'p394']

In [ ]:
all_subjects = {'genebody': genebody_subjects,
                'human36m': human36m_subjects,
                'zjumocap': zjumocap_subjects,}

In [ ]:
ntop_src = Path('/path/to/concatenated/NToP/datasets/')
subset = 'genebody'
posetype = 'coco'
for subj in genebody_subjects:
    print('GMT ' + time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(time.time())) + f' - Generating for {subset} : {subj}')
    kp_annos, bbox_annos = create_cocolike_pose_anno(ntop_src, subset, subj, posetype=posetype)

    pose_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_anno_{posetype}.json')
    with open(pose_anno_f, 'w') as f:
        json.dump(kp_annos, f)
    bbox_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_bbox.json')
    with open(bbox_anno_f, 'w') as f:
        json.dump(bbox_annos, f)

In [ ]:
subset = 'human36m'
for subj in human36m_subjects:
    print('GMT ' + time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(time.time())) + f' - Generating for {subset} : {subj}')
    kp_annos, bbox_annos = create_cocolike_pose_anno(ntop_src, subset, subj, posetype=posetype)

    pose_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_anno_{posetype}.json')
    with open(pose_anno_f, 'w') as f:
        json.dump(kp_annos, f)
    bbox_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_bbox.json')
    with open(bbox_anno_f, 'w') as f:
        json.dump(bbox_annos, f)   

In [ ]:
ntop_src = Path('/path/to/concatenated/NToP/datasets/')
subset = 'zjumocap'
posetype = 'coco'
for subj in zjumocap_subjects[:1]:
    print('GMT ' + time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(time.time())) + f' - Generating for {subset} : {subj}')
    kp_annos, bbox_annos = create_cocolike_pose_anno(ntop_src, subset, subj, posetype=posetype)

    pose_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_anno_{posetype}.json')
    with open(pose_anno_f, 'w') as f:
        json.dump(kp_annos, f)
    bbox_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_bbox.json')
    with open(bbox_anno_f, 'w') as f:
        json.dump(bbox_annos, f)

## Y.2 Hybrik-like annotation

In [ ]:
for img in images:
    img_id_str = f"00{img['id']}"
    if img_id_str not in raw_annos_everything.keys():
        print(f'{image_id_str} not found in raw annos')
        pass
    raw = raw_annos_everything[img_id_str]

    img['cam_idx'] = int(str(img['id'])[-1])
    img['cam_param'] = raw['cam_param']
    img['subject'] = subj
    img['subject_idx'] = int(str(dataset_idx)+subj_idx)
    img['action_name'] = raw['action_name']
    img['action_idx'] = raw['ac_idx']

for an in annotations:
    img_id_str = f"00{an['image_id']}"
    if img_id_str not in raw_annos_everything.keys():
        print(f'{image_id_str} not found in raw annos')
        pass
    raw = raw_annos_everything[img_id_str]
    
    an['thetas'] = raw['thetas']
    an['betas'] = raw['betas']
    an['root_coord'] = raw['root_coord']
    smpl_kps = raw['joints_3d']
    smpl_kps = [j[0:3] for j in smpl_kps]
    smpl_kps_flat = list(np.array(smpl_kps).flatten())
    an['smpl_keypoints'] = smpl_kps_flat
    del an['keypoints']
    del an['num_keypoints']


In [ ]:
from gen_cocolike_anno import create_hybrik_pose_anno

In [ ]:
ntop_src = Path('//path/to/concatenated/NToP/datasets/')
subset = 'genebody'
for subj in genebody_subjects[8:]:
    print('GMT ' + time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(time.time())) + f' - Generating for {subset} : {subj}')
    hybrik_annos = create_hybrik_pose_anno(ntop_src, subset, subj)

    hybrik_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_anno_hybrik.json')
    with open(hybrik_anno_f, 'w') as f:
        json.dump(hybrik_annos, f)

In [ ]:
subset = 'human36m'
for subj in human36m_subjects:
    print('GMT ' + time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(time.time())) + f' - Generating for {subset} : {subj}')
    hybrik_annos = create_hybrik_pose_anno(ntop_src, subset, subj)

    hybrik_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_anno_hybrik.json')
    with open(hybrik_anno_f, 'w') as f:
        json.dump(hybrik_annos, f)

In [ ]:
ntop_src = Path('/path/to/concatenated/NToP/datasets/')
subset = 'zjumocap'
for subj in zjumocap_subjects[0:1]:
    print('GMT ' + time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(time.time())) + f' - Generating for {subset} : {subj}')
    hybrik_annos = create_hybrik_pose_anno(ntop_src, subset, subj)

    hybrik_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_anno_hybrik.json')
    with open(hybrik_anno_f, 'w') as f:
        json.dump(hybrik_annos, f)

## Y.3. correct num_keypoints

In [ ]:
ntop_src = Path('/path/to/concatenated/NToP/datasets/')
subset = 'zjumocap'

for subj in all_subjects[subset]:
    coco_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_anno_coco.json')
    with open(coco_anno_f, 'r') as f:
        coco_anno = json.load(f)
    for an in coco_anno['annotations']:
        an['num_keypoints'] = 13
    with open(coco_anno_f, 'w') as f:
        json.dump(coco_anno, f)

## Y.4 Convert to xywh bboxes

In [ ]:
ntop_src = Path('/path/to/concatenated/NToP/datasets/')
subset = 'zjumocap'
subj = 'p313'
coco_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_anno_coco.json')
with open(coco_anno_f, 'r') as f:
    coco_anno = json.load(f)
for an in coco_anno['annotations']:
    bbox = an['bbox']
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    bbox[2] = w
    bbox[3] = h
coco_anno_xywh_f = Path(ntop_src, subset, 'anno', f'{subj}_anno_coco_xywh.json')
with open(coco_anno_xywh_f, 'w') as f:
    json.dump(coco_anno, f)


In [ ]:
ntop_src = Path('/path/to/concatenated/NToP/datasets/')
subset = 'genebody'

for subj in all_subjects[subset]:
    coco_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_anno_coco.json')
    with open(coco_anno_f, 'r') as f:
        coco_anno = json.load(f)
    for an in coco_anno['annotations']:
        bbox = an['bbox']
        w = bbox[2] - bbox[0]
        h = bbox[3] - bbox[1]
        bbox[2] = w
        bbox[3] = h
    coco_anno_xywh_f = Path(ntop_src, subset, 'anno', f'{subj}_anno_coco_xywh.json')
    with open(coco_anno_xywh_f, 'w') as f:
        json.dump(coco_anno, f)

In [ ]:
ntop_src = ntop_src = Path('/path/to/concatenated/NToP/datasets/')
subset = 'genebody'

for subj in all_subjects[subset]:
    bbox_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_bbox.json')
    with open(bbox_anno_f, 'r') as f:
        bbox_anno = json.load(f)
    for bb in bbox_anno:
        bbox = bb['bbox']
        w = bbox[2] - bbox[0]
        h = bbox[3] - bbox[1]
        bbox[2] = w
        bbox[3] = h
    bbox_xywh_f = Path(ntop_src, subset, 'anno', f'{subj}_bbox_xywh.json')
    with open(bbox_xywh_f, 'w') as f:
        json.dump(bbox_anno, f)

# Z. Final dataset concatenation

H36M renders: 470169
 7 subjects, 15 actions  - 5 sub for training 72%
    human36m:S1 - 55071 annotations
    human36m:S5 - 84888 annotations
    human36m:S6 - 56034 annotations
    human36m:S7 - 92331 annotations
    human36m:S8 - 58905 annotations
    human36m:S9 - 70281 annotations
    human36m:S11 - 52659 annotations

ZJU_Mocap renders: 27000
 9 subjects - 7 subjects for training
    zjumocap:p313 - 3600 annotations
    zjumocap:p315 - 7200 annotations
    zjumocap:p377 - 2052 annotations
    zjumocap:p386 - 1944 annotations
    zjumocap:p387 - 1944 annotations
    zjumocap:p390 - 4230 annotations
    zjumocap:p392 - 1944 annotations
    zjumocap:p393 - 2376 annotations
    zjumocap:p394 - 1710 annotations

genebody renders: 72900
 27 subjects - 19 subjects for training
    (each) - 2700 annotations


In [ ]:
genebody_subjects = ['ahha', 'alejandro', 'anastasia', 'aosilan', 'arslan',
                     'barlas', 'barry', 'camilo', 'dannier', 'gaoxing',
                     'huajiangtao5', 'joseph', 'kamal_ejaz', 'kemal',
                     'lihongyun', 'natacha', 'quyuanning', 'rabbi', 'rivera',
                     'songyujie', 'sunyuxing', 'wuwenyan', 'xujiarui',
                     'zhanghao', 'zhanghongwei', 'zhangziyu', 'zhuna2']
human36m_subjects = ['S1', 'S5', 'S6', 'S7', 'S8', 'S9', 'S11']
zjumocap_subjects = ['p313', 'p315', 'p377', 'p386', 'p387', 'p390', 'p392', 'p393', 'p394']

all_subjects = {'human36m': human36m_subjects,
                'genebody': genebody_subjects,
                'zjumocap': zjumocap_subjects}

ntop_src = Path('/path/to/concatenated/NToP/datasets/')

In [ ]:
ntop_train = [('human36m', subj) for subj in human36m_subjects[:5]]
ntop_train += [('genebody', subj) for subj in genebody_subjects[:19]]
ntop_train += [('zjumocap', subj) for subj in zjumocap_subjects[:7]]

ntop_val = [('human36m', subj) for subj in human36m_subjects[5:]]
ntop_val += [('genebody', subj) for subj in genebody_subjects[19:]]
ntop_val += [('zjumocap', subj) for subj in zjumocap_subjects[7:]]

print(ntop_val)

In [ ]:
def gen_ntop_final_annos(split='train', subset=None, bbox_type='xywh', anno_type='coco'):
    if split == 'train':
        anno_set = ntop_train
        if subset == 'human36m':
            anno_set = anno_set[:5]
        elif subset == 'genebody':
            anno_set = anno_set[5:24]
        elif subset == 'zjumocap':
            anno_set = anno_set[24:]
    elif split == 'val':
        anno_set = ntop_val
        if subset == 'human36m':
            anno_set = anno_set[:2]
        elif subset == 'genebody':
            anno_set = anno_set[2:10]
        elif subset == 'zjumocap':
            anno_set = anno_set[10:]

    annotations = []
    images = []
    bboxes = []
    
    for subset, subj in anno_set:
        if anno_type == 'hybrik':
            input_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_anno_{anno_type}.json')
        else:
            input_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_anno_{anno_type}_{bbox_type}.json')
        with open(input_anno_f, 'r') as f:
            input_anno = json.load(f)
        annotations += input_anno['annotations']
        images += input_anno['images']
        
        if anno_type != 'hybrik':
            bbox_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_bbox_{bbox_type}.json')
            with open(bbox_anno_f, 'r') as f:
                bbox_anno = json.load(f)
            bboxes += bbox_anno
            
        
    input_anno['images'] = images
    input_anno['annotations'] = annotations

    return input_anno, bboxes

In [ ]:
bbox_type = 'xywh'
anno_type = 'coco'

for split in ['val']:
    for subset in ['ntop', 'human36m', 'genebody', 'zjumocap']:
        kps_annos, bboxes = gen_ntop_final_annos(split, subset)
        
        with open(Path(ntop_src, 'ntop', 'annotations', f'{subset}_{split}_keypoints_{bbox_type}.json'), 'w') as f:
            json.dump(kps_annos, f)

        with open(Path(ntop_src, 'ntop', 'annotations', f'{subset}_{split}_bboxes_{bbox_type}.json'), 'w') as f:
            json.dump(bboxes, f)

In [ ]:
anno_type = 'hybrik'
for split in ['val', 'train']:
    for subset in ['ntop', 'human36m', 'genebody', 'zjumocap']:
        print(f'Concatenate annotation for {subset}_{split}')
        kps_annos, _ = gen_ntop_final_annos(split, subset, anno_type=anno_type)
        with open(Path(ntop_src, 'ntop', 'annotations', f'{subset}_{split}_{anno_type}_xyxy.json'), 'w') as f:
            json.dump(kps_annos, f)

In [ ]:
anno_type = 'hybrik'
for split in ['val', 'train']:
    for subset in ['ntop', 'human36m', 'genebody', 'zjumocap']:
        anno_f = Path(ntop_src, 'ntop', 'annotations', f'{subset}_{split}_{anno_type}_xyxy.json')
        with open(anno_f, 'r') as f:
            annos = json.load(f)
        for an in annos['annotations']:
            bbox = an['bbox']
            w = bbox[2] - bbox[0]
            h = bbox[3] - bbox[1]
            bbox[2] = w
            bbox[3] = h
        anno_xywh_f = Path(ntop_src, 'ntop', 'annotations', f'{subset}_{split}_{anno_type}_xywh.json')
        with open(anno_xywh_f, 'w') as f:
            json.dump(annos, f)

In [ ]:
anno_type = 'hybrik'
split = 'val'
subset = 'genebody'
kps_annos, bboxes = gen_ntop_final_annos(split, subset, anno_type=anno_type)

print(kps_annos['annotations'][0])

In [ ]:
annotations = []
images = []
for subset, subj in ntop_val:
    coco_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_anno_coco_xywh.json')
    with open(coco_anno_f, 'r') as f:
        coco_anno = json.load(f)
    annotations += coco_anno['annotations']
    images += coco_anno['images']
        
coco_anno['images'] = images
coco_anno['annotations'] = annotations

with open(Path(ntop_src, 'ntop', 'annotations', f'ntop_val_keypoints_xywh.json'), 'w') as f:
    json.dump(coco_anno, f)

In [ ]:
annotations = []
images = []
for subset, subj in ntop_val:
    hybrik_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_anno_hybrik.json')
    with open(hybrik_anno_f, 'r') as f:
        hybrik_anno = json.load(f)
    annotations += hybrik_anno['annotations']
    images += hybrik_anno['images']
        
hybrik_anno['images'] = images
hybrik_anno['annotations'] = annotations

with open(Path(ntop_src, 'ntop', 'annotations', 'ntop_val_hybrik.json'), 'w') as f:
    json.dump(hybrik_anno, f)

In [ ]:
bboxes = []
for subset, subj in ntop_train:
    bbox_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_bbox_xywh.json')
    with open(bbox_anno_f, 'r') as f:
        bbox_anno = json.load(f)
    bboxes += bbox_anno

with open(Path(ntop_src, 'ntop', 'annotations', f'ntop_train_bboxes_xywh.json'), 'w') as f:
    json.dump(bboxes, f)

In [ ]:
# move rgb renders to train and val folder
for subset, subj in ntop_train:
    rgb_dir = f"{ntop_src}/{subset}/concat/{subj}/rgb/"
    dst_dir = f"{ntop_src}/ntop/train_images/"
    imgs = os.listdir(rgb_dir)
    for im in imgs:
        shutil.copy(rgb_dir+im, dst_dir+im)

# A. Visualization

In [ ]:
from gen_cocolike_anno import vis_skeleton_single_image

In [ ]:
ntop_src = Path('/path/to/concatenated/NToP/datasets/')
subset = 'genebody'
subj='natacha'
posetype='coco'
pose_anno_f = Path(ntop_src, subset, 'anno', f'{subj}_anno_{posetype}_xyxy.json')
with open(pose_anno_f, 'r') as f:
    kp_annos = json.load(f)

In [ ]:
ids = []
for anno in kp_annos['annotations'][1134:1143]:
    ids.append(anno['id'])

In [ ]:
print(ids)

In [ ]:
ids = [int(f'2016000{i:06d}12106') for i in range(150)]

In [ ]:
ids = [int(f'20160000000661005{i}') for i in range(9)]

In [ ]:
print(ids)

In [ ]:
ntop_src = Path('/path/to/concatenated/NToP/datasets/')
pose_anno_f = Path(ntop_src, 'genebody', 'hybrik_anno', 'genebody_train_hybrik_xyxy.json')
with open(pose_anno_f, 'r') as f:
    kp_annos = json.load(f)

In [ ]:
annos = [an  for an in kp_annos['annotations'] if an['id'] in ids]

In [ ]:
annos[0]

In [ ]:
for anno in annos:
    kps = anno['smpl_joints_img']
    kps2d = [[kps[i*2], kps[i*2+1]] for i in range(24)]
    
    bbox = anno['bbox']
    
    #img_f = Path(ntop_src, subset, 'concat', subj, 'rgb', f'00{anno["image_id"]}.png')
    img_id = anno['image_id']
    #img_id += 102050
    img_f = Path(Path('/mnt/data/yjin/NTOP/'), 'ntop', 'train_images', f'00{img_id}.png')
    #vis_skeleton_single_image(str(img_f.resolve()), kps2d, bbox)
    #fig = vis_skeleton_single_image(kps2d, str(img_f.resolve()))
    fig = vis_skeleton_single_image(kps2d)
    #skel_f = Path(f'/home/yjin/ntop_2d_fig/{img_id}.png')
    #fig.savefig(skel_f)

In [ ]:
from gen_cocolike_anno import plotSkel2D, skeleton_tree

def vis_skeleton_single_image(keypoints, image_path=None, bbox=None):
    kpts2d = np.array(keypoints)

    fig = plt.figure(figsize =[5,5])
    ax = fig.add_subplot(111)
    if image_path is not None:
        img = cv2.imread(image_path)
        #ax.imshow(img[..., ::-1])
        H, W = img.shape[:2]
    else:
        H, W = 1000, 1000
#    plotSkel2D(kpts2d, ax = ax)
    plotSkel2D(kpts2d, skeleton_tree, ax, linewidth=2, alpha=1, max_range=1, thres=0.5)
    if bbox is not None:
        plot_bbox(bbox, ax)
    
    return fig

In [ ]:
from gen_cocolike_anno import skeleton_tree, skel3dplot

In [ ]:
import torch
import matplotlib.pyplot as plt

def skel3dplot(kps, config, ax = None, phi = 0, theta = 0, linewidth = 4, color = None, max_range = 1):
    if kps.shape[1] == 17:
        kps = kps.reshape(17,3)
    else:
        kps = kps.reshape(24,3)
    kps = kps[:, [0,2,1]]
    multi = False
    if torch.is_tensor(kps):
        if len(kps) == 3:
            print(">>> View Multiperson")
            multi = True
            if kps.shape[1] != 3:
                kps = kps.transpose(1,2)
        elif len(kps) == 2:
            if kps.shape[0] != 3:
                kps = kps.transpose(0,1)
        else:
            raise RuntimeError('Wrong shapes for Kps')
    else:
        if kps.shape[0] != 3:
            kps = kps.T
    # kps: bn, 3, NumOfPoints or (3, N)

    if ax is None:
        print("Creating figure >>> ")
        fig = plt.figure(figsize =[10,10])
        ax = fig.add_subplot(111, projection = '3d')

    if kps.shape[1] == 17:
        for idx, (i,j) in enumerate(config['coco_tree']):
            if multi:
                for b in range(kps.shape[0]):
                    ax.plot([kps[b][0][i], kps[b][0][j]],
                            [kps[b][1][i], kps[b][1][j]],
                            [kps[b][2][i], kps[b][2][j]],
                            lw=linewidth,
                            color=config['color'][idx] if color is None else color,
                            alpha=1)
            else:
                ax.plot([kps[0][i], kps[0][j]], [kps[1][i], kps[1][j]],
                        [kps[2][i], kps[2][j]],
                        lw=linewidth,
                        color=config['color'][idx],
                        alpha=1)
    else:
        for idx, (i,j) in enumerate(config['smpl_tree']):
            if multi:
                for b in range(kps.shape[0]):
                    ax.plot([kps[b][0][i], kps[b][0][j]],
                            [kps[b][1][i], kps[b][1][j]],
                            [kps[b][2][i], kps[b][2][j]],
                            lw=linewidth,
                            color=config['smpl_color'][idx] if color is None else color,
                            alpha=1)
            else:
                ax.plot([kps[0][i], kps[0][j]], [kps[1][i], kps[1][j]],
                        [kps[2][i], kps[2][j]],
                        lw=linewidth,
                        color=config['smpl_color'][idx],
                        alpha=1)    
    
    if multi:
        for b in range(kps.shape[0]):
            ax.scatter(kps[b][0], kps[b][1], kps[b][2], color = 'r', alpha = 1,  marker='o')
            #for joint_idx, (x, y, z) in enumerate(zip(kps[b][0], kps[b][1], kps[b][2])):
            #    ax.text(x, y, z, str(joint_idx), fontsize=8, color='black', va='center', ha='center')
    else:
        ax.scatter(kps[0], kps[1], kps[2], color = 'r', s = 100,  marker='o')
        #for joint_idx, (x, y, z) in enumerate(zip(kps[0], kps[1], kps[2])):
         #   ax.text(x, y, z, str(joint_idx), fontsize=8, color='black', va='center', ha='center')

    ax.view_init(phi, theta)
    ax.set_xlim(-0.4, max_range)
    ax.set_ylim(-0.2, max_range)
    ax.set_zlim(-max_range, 0.4)
    ax.set_box_aspect((1.,1.0,1.))
    
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_zticklabels([])


    return fig, ax
